Memoria TFG. Tema 4 - Patrones de organización de datos: Estructuración jerárquica. 

Este ejemplo de patrón de organización de datos consiste en que dada una estructura de datos en formato tabla, ésta es convertida a otro tipo de estructura en un formato jerarquizado como puede ser JSON o XML. La ejecución se deriva a un archivo; supera la capacidad de muestra de Jupyter Notebook.

In [4]:
%%writefile jerarquico.py
#!/usr/bin/env python
#Usamos el archivo foros.csv

from mrjob.job import MRJob
import xmlify
import json
#import sys, os, re

class jerarquico(MRJob):
    
    def mapper(self,_, line):
        linea=line.split(";") # Cada línea es un mensaje del foro (pregunta, respuesta o comentario)
        
        mensaje=linea[4] # Recogemos el mensaje de la posición 4 de la línea
        tipoMensaje=linea[5] #Recogemos de la posición 5, si es una pregunta, respuesta o comentario
        #Para poder agrupar todos los mensajes de una pregunta, usamos el valor[0] que indica el identificador
        #único y el valor[7] que almacena el identificador de la pregunta (valor[0]) a la que está asociada
        #esto es fundamental por que hará que en el reducer agrupe todos los elementos relacionados con una pregunta
        
        if tipoMensaje=="question": 
            idMensaje=linea[0] #Almacenamos el id único del mensaje
            yield idMensaje,(tipoMensaje,mensaje)
        else:
            idMensaje=linea[7] #Almacenamos el identificador del mensaje idMensaje  
            yield idMensaje,(tipoMensaje,mensaje)
     
    def reducer(self, key, values):
        #diccionario=dict() #Para el caso que usemos diccionarios
        listaValores=[]
        listaPrincipal=[]
        listaAuxiliar=[] 
        
        for v in values: #Metemos los valores que vienen en un matriz
            listaValores.append(v) #Matriz que contiene el tipo de mensaje y el mensaje asociado 
        
        for valor in listaValores:
            if valor[0]=="question":#Si es una pregunta la metemos en la lista principal
                listaPrincipal.append(valor[1])
            else:
                listaAuxiliar.append(valor[1]) # Si son respuestas, las vamos agregando a una lista
        
        listaPrincipal.append(listaAuxiliar) #agregamos la lista de respuestas a la lista principal
        #La idea es que quede la siguiente estructura
        # key	["pregunta", ["respuesta", "respuesta"]]
        #Esta estructura se la pasamos a la funcion dumps y la convierte automáticamente a XML o JSON
        
        #diccionario[key]=listaPrincipal #Para el caso que usemos diccionarios
        
        yield key,xmlify.dumps(listaPrincipal,root = key) #Conversion a XML indicando en el raiz el id del mensaje
        #yield key,xmlify.dumps(diccionario) # Conversion a XML para el caso que usemos diccionarios
       
        
       
        
if __name__ == '__main__':
    jerarquico.run()

Writing jerarquico.py


In [6]:
!python jerarquico.py archivos_datos/foros.csv > jerarquico.xml

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\cyber\AppData\Local\Temp\jerarquico.manuel.20201125.091856.211355
Running step 1 of 1...
job output is in C:\Users\cyber\AppData\Local\Temp\jerarquico.manuel.20201125.091856.211355\output
Streaming final output from C:\Users\cyber\AppData\Local\Temp\jerarquico.manuel.20201125.091856.211355\output...
Removing temp directory C:\Users\cyber\AppData\Local\Temp\jerarquico.manuel.20201125.091856.211355...


Memoria PFG. Tema 4 - Patrones de organización de datos. Estructuración jerárquica. 

Esta segunda versión vamos a usar la estructura diccionarios de Python

In [2]:
%%writefile jerarquicoV2.py
#!/usr/bin/env python
#Usamos el archivo foros.csv

from mrjob.job import MRJob
import xmlify
import json
import sys, os, re

class jerarquicoV2(MRJob):
    
    def mapper(self,_, line):
        linea=line.split(";") # Cada línea es un mensaje del foro (pregunta, respuesta o comentario)
        
        mensaje=linea[4] # Recogemos el mensaje de la posición 4 de la línea
        tipoMensaje=linea[5] #Recogemos de la posición 5, si es una pregunta, respuesta o comentario
        #Para poder agrupar todos los mensajes de una pregunta, usamos el valor[0] que indica el identificador
        #único y el valor[7] que almacena el identificador de la pregunta (valor[0]) a la que está asociada
        #esto es fundamental por que hará que en el reducer agrupe todos los elementos relacionados con una pregunta
        
        if tipoMensaje=="question": 
            idMensaje=linea[0] #Almacenamos el id único del mensaje
            yield idMensaje,(tipoMensaje,mensaje)
        else:
            idPadre=linea[7] #Almacenamos el identificador del mensaje idMensaje  
            yield idPadre,(tipoMensaje,mensaje)
     
    def reducer(self, key, values):
        diccionario=dict() #Para el caso que usemos diccionarios
        matrizParaXML=[]
        listaPrincipal=[]
        listaAuxiliar=[]
        
        for v in values: #Metemos los valores que vienen en un matriz
            matrizParaXML.append(v) #Matriz que contiene el tipo de mensaje y el mensaje asociado 
        
        for valor in matrizParaXML:
            if valor[0]=="question":#Si es una pregunta la metemos en la lista principal
                listaPrincipal.append(valor[1])
            else:
                listaAuxiliar.append(valor[1]) # Si son respuestas, las vamos agregando a una lista
        
        listaPrincipal.append(listaAuxiliar) #agregamos la lista de respuestas a la lista principal
        #La idea es que quede la siguiente estructura
        # key	["pregunta", ["respuesta", "respuesta"]]
        #Esta estructura se la pasamos a la funcion dumps y la convierte automáticamente a XML o JSON
        
        diccionario[key]=listaPrincipal #Para el caso que usemos diccionarios
        
        yield key,xmlify.dumps(diccionario) # Conversion a XML para el caso que usemos diccionarios
        
        #Habría que unificar todos los mensajes, ya que salen solo uno por salida del Reducer.
        
if __name__ == '__main__':
    jerarquicoV2.run()

Writing jerarquicoV2.py


In [3]:
!python jerarquicoV2.py archivos_datos/foros.csv

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



"8040"	"<data><n8040><n8040-item>&lt;p&gt;In Unit 2.29 the video is missing.&lt;/p&gt; &lt;p&gt;Is this a bug or the video is not posted yet???&lt;/p&gt;</n8040-item><n8040-item><n8040-item-item>&lt;p&gt;Works fine for us. Refresh your browser window (right click, reload in Chrome)&lt;/p&gt;</n8040-item-item><n8040-item-item>&lt;p&gt;Nope, it didn't work and neither clearing the browser history!!!&lt;/p&gt;</n8040-item-item><n8040-item-item>&lt;p&gt;Could be your local ISP's Google caching. Wait an hour and see what happens&lt;/p&gt;</n8040-item-item><n8040-item-item>&lt;p&gt;same thing here, no 29.  You could get to the quiz though.&lt;/p&gt;</n8040-item-item><n8040-item-item>\"&lt;p&gt;Since you can access the video, &lt;a href=\"\"http://forums.udacity.com/users/100005361/peterudacity\"\"&gt;@Peter&lt;/a&gt; could you link us to it on youtube?&lt;/p&gt;\"</n8040-item-item><n8040-item-item>&lt;p&gt;Just in case this is relevant information, I am also having this problem and the video

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\cyber\AppData\Local\Temp\jerarquicoV2.manuel.20201125.091607.673446
Running step 1 of 1...
job output is in C:\Users\cyber\AppData\Local\Temp\jerarquicoV2.manuel.20201125.091607.673446\output
Streaming final output from C:\Users\cyber\AppData\Local\Temp\jerarquicoV2.manuel.20201125.091607.673446\output...
Removing temp directory C:\Users\cyber\AppData\Local\Temp\jerarquicoV2.manuel.20201125.091607.673446...
